In [1]:
from __future__ import print_function, division
%load_ext autoreload

In [2]:
%autoreload 2

import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter


from src.model.mmd_grud_utils import *
from utils.eda_functions import (load_from_pickle, save_to_pickle)
from config import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
BLINDED = False
RANDOM = 0

In [8]:
RESAMPLED_DIR = '../data/resampled/'

In [9]:
X_train = load_from_pickle(os.path.join(RESAMPLED_DIR, 'vitals_train.pkl'))
X_dev = load_from_pickle(os.path.join(RESAMPLED_DIR, 'vitals_dev.pkl'))
X_test = load_from_pickle(os.path.join(RESAMPLED_DIR, 'vitals_test.pkl'))

Ys_train = load_from_pickle(os.path.join(RESAMPLED_DIR, 'Ys_train.pkl'))
Ys_dev = load_from_pickle(os.path.join(RESAMPLED_DIR, 'Ys_dev.pkl'))
Ys_test = load_from_pickle(os.path.join(RESAMPLED_DIR, 'Ys_test.pkl'))

In [ ]:
# X_flat = X_train.reset_index()
# X_flat.columns = X_flat.columns.droplevel(-1)
# Y_flat = Ys_train.reset_index()
# expanded_Y = pd.merge(X_flat, Y_flat, on=['subject_id', 'hadm_id', 'icustay_id'], how='left')
# Y_train = expanded_Y[['subject_id', 'hadm_id', 'icustay_id', 'hours_in', 'mort_hosp']]
# Y_train.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in'], inplace=True)
# 
# X_flat = X_dev.reset_index()
# X_flat.columns = X_flat.columns.droplevel(-1)
# Y_flat = Ys_dev.reset_index()
# expanded_Y = pd.merge(X_flat, Y_flat, on=['subject_id', 'hadm_id', 'icustay_id'], how='left')
# Y_dev = expanded_Y[['subject_id', 'hadm_id', 'icustay_id', 'hours_in', 'mort_hosp']]
# Y_dev.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in'], inplace=True)
# 
# X_flat = X_test.reset_index()
# X_flat.columns = X_flat.columns.droplevel(-1)
# Y_flat = Ys_test.reset_index()
# expanded_Y = pd.merge(X_flat, Y_flat, on=['subject_id', 'hadm_id', 'icustay_id'], how='left')
# Y_test = expanded_Y[['subject_id', 'hadm_id', 'icustay_id', 'hours_in', 'mort_hosp']]
# Y_test.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in'], inplace=True)

In [ ]:
# del Ys_train, Ys_dev, Ys_test, expanded_Y, X_flat, Y_flat

In [ ]:
# all_cols = []
# for i, col in enumerate(X_train.columns):
#     if col[0] not in all_cols:
#         all_cols.append(col[0])
#         print(col[0])

In [10]:
GAP_TIME          = 0  # In hours
WINDOW_SIZE       = 48 # In hours
SEED              = 69
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
GPU               = '2'

# os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
torch.manual_seed(SEED)

In [11]:
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out

class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

### Task Prediction

#### Hyperparameters

In [16]:
N = 10

# GRU_D_dist = DictDist({
#     'cell_size': ss.randint(50, 75),
#     'hidden_size': ss.randint(65, 95),
#     'learning_rate': ss.uniform(2e-3, 1e-1),
#     'num_epochs': ss.randint(15, 150),
#     'patience': ss.randint(3, 7),
#     'batch_size': ss.randint(35, 65),
#     'early_stop_frac': ss.uniform(0.05, 0.1),
#     'seed': ss.randint(1, 10000),
# })

EARLY_STOP_FRAC = 0.05
SEED = 69

GRU_D_dist = DictDist({
    # 'cell_size': ss.randint(50, 75),
    'hidden_size': ss.randint(65, 95),
    'learning_rate': ss.loguniform(1e-4, 1e-1),
    'num_epochs': ss.randint(15, 150),
    'patience': ss.randint(3, 20),
    'batch_size': ss.randint(35, 65),
    # 'early_stop_frac': 0.1,  # Or any specific value or narrow range
    # 'seed': 69,  # Or any fixed seed value
    # 'grad_clip_value': ss.uniform(0.5, 1.0)
})

np.random.seed(SEED)
GRU_D_hyperparams_list = GRU_D_dist.rvs(N)

# with open('../src/model/baselines_gru-d.pkl', mode='rb') as f: results = pickle.load(f)

In [24]:
GRU_D_hyperparams_list = [
    {'hidden_size': 128, 'learning_rate': 0.0001, 'num_epochs': 100, 'patience': 20, 'batch_size': 32},
    {'hidden_size': 64, 'learning_rate': 0.0001, 'num_epochs': 100, 'patience': 20, 'batch_size': 64},
    {'hidden_size': 64, 'learning_rate': 0.0005, 'num_epochs': 100, 'patience': 10, 'batch_size': 64},
    {'hidden_size': 128, 'learning_rate': 0.0005, 'num_epochs': 100, 'patience': 15, 'batch_size': 32},
    {'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 100, 'patience': 15, 'batch_size': 128},
    {'hidden_size': 64, 'learning_rate': 0.001, 'num_epochs': 100, 'patience': 10, 'batch_size': 64},
    {'hidden_size': 128, 'learning_rate': 0.001, 'num_epochs': 100, 'patience': 10, 'batch_size': 32},
    {'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 100, 'patience': 15, 'batch_size': 128},
    {'hidden_size': 64, 'learning_rate': 0.001, 'num_epochs': 100, 'patience': 10, 'batch_size': 64},
    {'hidden_size': 32, 'learning_rate': 0.01, 'num_epochs': 100, 'patience': 5, 'batch_size': 128}
]

In [3]:
from itertools import product

# Define the parameter grid
hidden_sizes = [32, 64, 128]
learning_rates = [0.0001, 0.0005, 0.001, 0.01]
batch_sizes = [32, 64, 128]

# Placeholder for storing the results
grid_search_results = []

# The fixed number of epochs for training
num_epochs = 100
patience = 10

# Iterate over all combinations of hyperparameters
for learning_rate, hidden_size, batch_size in product(hidden_sizes, learning_rates, batch_sizes):

    grid_search_results.append({
        'hidden_size': hidden_size,
        'learning_rate': learning_rate,
        'batch_size': batch_size,
        'patience': patience,
        'num_epochs': num_epochs
    })

In [4]:
len(grid_search_results)

36

In [25]:
results = {}

#### GRU-D

In [26]:
from src.model.mmd_grud_utils import *

In [ ]:
# hyperparams = {'cell_size': 72,
#  'hidden_size': 128,
#  'learning_rate': 0.0001,
#  'num_epochs': 20,
#  'patience': 10,
#  'batch_size': 64}

In [ ]:
model_name       = 'GRU-D'
tasks             = ['mort_hosp']
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False

if model_name not in results: results[model_name] = {}

# for t in tasks:         # Edit - DR
#     if t not in results[model_name]: results[model_name][t] = {}
t = 'mort_hosp'
if t not in results[model_name]: results[model_name][t] = {}
    # for n, X_train, X_dev, X_test in (('full_X', X_train, X_dev, X_test),):  # Edit - DR
n, X_train, X_dev, X_test = ('full_X', X_train, X_dev, X_test)
print("Running model %s on target %s with representation %s" % (model_name, t, n))
X_mean = np.nanmean(
    to_3D_tensor(
        X_train.loc[:, pd.IndexSlice[:, 'mean']] *
        np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
    ),
    axis=0, keepdims=True
).transpose([0, 2, 1])
base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}

if n in results[model_name][t]:
    if not RERUN:
        print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
        print(results[model_name][t][n])
        # continue  # Edit - DR
    best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
    print("Loading best hyperparams", best_hyperparams)
else:
    best_s, best_hyperparams = -np.Inf, None
    for i, hyperparams in enumerate(hyperparams_list):
        print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

        # early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]    # Edit - DR
        batch_size = hyperparams['batch_size']

        np.random.seed(SEED)
        all_train_subjects = list(
            np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
        )
        N_early_stop        = int(len(all_train_subjects) * EARLY_STOP_FRAC)
        train_subjects      = all_train_subjects[:-N_early_stop]
        early_stop_subjects = all_train_subjects[-N_early_stop:]
        X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
        Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

        X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
        Ys_train_early_stop = Ys_train[
            Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
        ]

        train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
        early_stop_dataloader = prepare_dataloader(
            X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size
        )
        dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=batch_size)
        test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

        model_hyperparams = copy.copy(base_params)
        model_hyperparams.update(
            {k: v for k, v in hyperparams.items() if k in ('hidden_size', 'batch_size')}
        )
        # del model_hyperparams['cell_size']  # Edit - DR
        model = GRUD(**model_hyperparams)
        
        best_model, _ = Train_Model(
            model, train_dataloader, early_stop_dataloader,
            **{k: v for k, v in hyperparams.items() if k in (
                'num_epochs', 'patience', 'learning_rate', 'batch_size'
            )}
        )

        probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
        probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
        labels_dev        = np.concatenate(labels_dev)
        s = roc_auc_score(labels_dev, probabilities_dev)
        if s > best_s:
            best_s, best_hyperparams = s, hyperparams
            print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))

Running model GRU-D on target mort_hosp with representation full_X
On sample 1 / 10 (hyperparams = {'hidden_size': 128, 'learning_rate': 0.0001, 'num_epochs': 100, 'patience': 20, 'batch_size': 32})
Model Structure:  GRUD(
  (zl): Linear(in_features=206, out_features=128, bias=True)
  (rl): Linear(in_features=206, out_features=128, bias=True)
  (hl): Linear(in_features=206, out_features=128, bias=True)
  (gamma_x_l): FilterLinear(in_features=39, out_features=39, bias=True)
  (gamma_h_l): Linear(in_features=39, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type determined by the model
Epoch: 0, train_loss: 0.74328144, valid_loss: 0.78043734, time: [82.43], best model: 1
Epoch: 1, train_loss: 0.63497225, valid_loss: 0.65863192, time: [73.11], best model: 1
Epoch: 2, train_loss: 0.57226097, val

In [ ]:
## Test
np.random.seed(SEED)
hyperparams = best_hyperparams # In case I forgot a replace below
# early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]

X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))

all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
N_early_stop = int(len(all_train_subjects) * EARLY_STOP_FRAC)
train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

model_hyperparams = copy.copy(base_params)
model_hyperparams.update(
    {k: v for k, v in best_hyperparams.items() if k in ('hidden_size', 'batch_size')}
)
model = GRUD(**model_hyperparams)


best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
    model, train_dataloader, early_stop_dataloader,
    **{k: v for k, v in best_hyperparams.items() if k in (
        'num_epochs', 'patience', 'learning_rate', 'batch_size'
    )}
)

probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

y_score = np.concatenate(probabilities_test)[:, 1]
y_pred  = np.argmax(probabilities_test)
y_true  = np.concatenate(labels_test)

auc   = roc_auc_score(y_true, y_score)
auprc = average_precision_score(y_true, y_score)
acc   = accuracy_score(y_true, y_pred)
F1    = f1_score(y_true, y_pred)
print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
print(auc, auprc, acc, F1)

results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
with open('../src/model/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)